In [1]:
import wabi
from wabi import graphql2pay

In [2]:
import json
import requests as req
import pandas as pd

In [3]:
from wabi import sql_wabilytics, maps_2b

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wab

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)


In [4]:
stores_wallets = """
SELECT 
	s.wabipay_account
FROM wabiRojo.stores s 
WHERE s.wabipay_account IS NOT NULL
LIMIT 4;
"""

In [5]:
stores_df = sql_wabilytics.querySQL(stores_wallets,'wabiRojo')

In [6]:
stores_df

,wabipay_account
0,ws.padua.m
1,ws.ruasaojorge
2,ws.caetanogarbelotto
3,ws.palomar.cap.rosales


In [7]:
graphql2pay.bypassToken()
access_token = graphql2pay.token

Got it!


In [8]:
def query_maker(i,token):
    
    query = '''
    query{
  findUserByUsername(input:{
    username:"'''+stores_df.loc[i,'wabipay_account']+'''"
    accessToken:"'''+token+'''"
  }){
    ... on User{
      username
      wallet{
        ... on Wallet{
          name
          id
          
          
        }
        
      }
    }
  }
}
    '''
    return query

In [9]:
results = [] #esto será una lista de diccionarios
url = 'https://api.prod.yopago.app/graphql'
for i in range(0,len(stores_df)):
    query = query_maker(i,access_token)
    response = req.post(url = url, json = {"query":query})
    results.append(json.loads(response.content))

In [10]:
results

[{'extensions': {'duration': 15788548},
  'data': {'findUserByUsername': {'username': 'ws.padua.m',
    'wallet': {'name': 'ws.padua.m',
     'id': 'http://backend.yopago.local/wallet/8b07ad9c-c4d2-412c-91ea-6d36ee95d6c9'}}}},
 {'extensions': {'duration': 17281803},
  'data': {'findUserByUsername': {'username': 'ws.ruasaojorge',
    'wallet': {'name': 'ws.ruasaojorge',
     'id': 'http://backend.yopago.local/wallet/b359abf5-17f6-4d0b-b1b1-e39aeca2e419'}}}},
 {'extensions': {'duration': 17589112},
  'data': {'findUserByUsername': {'username': 'ws.caetanogarbelotto',
    'wallet': {'name': 'ws.caetanogarbelotto',
     'id': 'http://backend.yopago.local/wallet/1938412c-0a4d-40b3-8a1e-fe95f8fe2f69'}}}},
 {'extensions': {'duration': 13159741},
  'data': {'findUserByUsername': {'username': 'ws.palomar.cap.rosales',
    'wallet': {'name': 'ws.palomar.cap.rosales',
     'id': 'http://backend.yopago.local/wallet/08c1966f-d3ab-4f34-b0d8-1d9459d9d006'}}}}]

In [11]:
i = 0
results_df = pd.DataFrame()
for json_result in results:
    if json_result['data']['findUserByUsername']:
        print(json_result['data']['findUserByUsername'])
        results_df.loc[i,'wabipay_account'] = stores_df.loc[i,'wabipay_account']
        results_df.loc[i,'username_graphQL'] = json_result['data']['findUserByUsername']['username']
        results_df.loc[i,'name_graphQL'] = json_result['data']['findUserByUsername']['wallet']['name']
        results_df.loc[i,'id_graphQL'] = json_result['data']['findUserByUsername']['wallet']['id']
        i+=1
        
        
            
            
    

{'username': 'ws.padua.m', 'wallet': {'name': 'ws.padua.m', 'id': 'http://backend.yopago.local/wallet/8b07ad9c-c4d2-412c-91ea-6d36ee95d6c9'}}
{'username': 'ws.ruasaojorge', 'wallet': {'name': 'ws.ruasaojorge', 'id': 'http://backend.yopago.local/wallet/b359abf5-17f6-4d0b-b1b1-e39aeca2e419'}}
{'username': 'ws.caetanogarbelotto', 'wallet': {'name': 'ws.caetanogarbelotto', 'id': 'http://backend.yopago.local/wallet/1938412c-0a4d-40b3-8a1e-fe95f8fe2f69'}}
{'username': 'ws.palomar.cap.rosales', 'wallet': {'name': 'ws.palomar.cap.rosales', 'id': 'http://backend.yopago.local/wallet/08c1966f-d3ab-4f34-b0d8-1d9459d9d006'}}


In [12]:
results_df

,wabipay_account,username_graphQL,name_graphQL
0,ws.padua.m,ws.padua.m,http://backend.yopago.local/wallet/8b07ad9c-c4...
1,ws.ruasaojorge,ws.ruasaojorge,http://backend.yopago.local/wallet/b359abf5-17...
2,ws.caetanogarbelotto,ws.caetanogarbelotto,http://backend.yopago.local/wallet/1938412c-0a...
3,ws.palomar.cap.rosales,ws.palomar.cap.rosales,http://backend.yopago.local/wallet/08c1966f-d3...
